In [1]:
import pandas as pd
from ekonlpy.sentiment import MPCK
mpck = MPCK()

In [2]:
hawk = pd.read_csv('c:/users/user/hawk_dic.csv')
dove = pd.read_csv('c:/users/user/dove_dic.csv')

In [3]:
minutes = pd.read_csv('c:/users/user/minutes_with_label.csv', index_col = 0).dropna().reset_index(drop=True)
# minutes['ngram'] = minutes.ngram.apply(lambda x: x.replace('[','').replace(']','').replace('\'','').replace(' ','').split(','))

의사록내의 개별 문장에 대한 hawkish/dovish features를 토대로 개별문장 tone을 구한 뒤,  
개별문장 톤을 종합해서 문서에 대한 tone을 구한다.

In [4]:
def tone_sentence(sentence):
    n_hawk, n_dove = 0, 0
    tokens = mpck.tokenize(sentence)
    ngrams = mpck.ngramize(tokens) + tokens
    for ngram in ngrams:
        n_hawk += list(hawk.ngram).count(ngram)
        n_dove += list(dove.ngram).count(ngram)
    try:     
        result =  (n_hawk - n_dove)/(n_hawk + n_dove)
    except:
        result = 0 # ZeroDivisionError 처리
        
    return result

def tone_minute(minute):
    tone = {'hawk':0,'dove':0}
    clean_minute = list(map(str.strip, minute.replace('  ','').split('.')))
    for sentence in clean_minute:
        if tone_sentence(sentence) > 0:
            tone['hawk'] += 1
        elif tone_sentence(sentence) < 0:
            tone['dove'] += 1
            
    return (tone['hawk'] - tone['dove']) / (tone['hawk'] + tone['dove'])

기준금리와 비교

In [ ]:
from tqdm.notebook import tqdm

df_result = minutes

for row, minute in enumerate(tqdm(minutes.content)):
    df_result.loc[row,'TONE_Doc'] = tone_minute(minute)


In [ ]:
df_result = df_result.drop(['title','content','ngram','label'], axis=1)
df_result.to_csv('c:/users/user/Tone_doc.csv', index = False)

In [6]:
df_result.head()

,date,TONE_Doc
0,2008-04-10,0.276923
1,2008-05-08,0.328467
2,2008-06-12,0.354839
3,2008-07-10,0.369048
4,2008-08-07,0.309353


In [ ]:
print('Hawkish: ',(df_result.TONE_Doc > 0).sum())
print('Dovish: ',(df_result.TONE_Doc < 0).sum())